In [1]:
import pandas as pd
import re
from rapidfuzz import process
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from collections import Counter
import os
# Set plotting style and directory
plt.style.use('ggplot')
sns.set_palette("colorblind")
os.makedirs('figures', exist_ok=True)

## Data Loading and Preprocessing

In [4]:
#########################################################
# PART 1: DATA LOADING AND PREPROCESSING

#The analysis begins by loading and merging the article and model datasets using the DOI as the primary key. The preprocessing includes:
  # - Data cleaning: Handling missing values, converting categorical fields, and addressing negative values in confidence intervals (which appear to be coding for missing data using -99)
  # - New indicator variables: Creating flags for demographic groups based on the 'compare' column:
    # - Black/African American comparisons
    # - Hispanic/Latino comparisons
    # - Asian/Pacific Islander comparisons
    # - Indigenous/Native American comparisons
    # - Missing data analysis: Visualizing patterns using missingno plots and determining whether imputation is appropriate
#########################################################

def load_data(article_path, model_path):
    """
    Load article and model datasets
    """
    print("Loading datasets...")
    article_df = pd.read_csv("article_dat.csv")
    model_df = pd.read_csv("model_dat.csv")

    print(f"Article dataset shape: {article_df.shape}")
    print(f"Model dataset shape: {model_df.shape}")

    return article_df, model_df

def explore_data(article_df, model_df):
    """
    Explore the raw datasets and report key insights
    """
    print("\n--- Data Exploration ---")

    # Explore article data
    print("\nArticle data - column types:")
    print(article_df.dtypes.value_counts())

    # Check health outcome-related columns
    health_cols = [col for col in article_df.columns if 'health' in col or 'cancer' in col or
                  'endo' in col or 'fibroid' in col or 'fert' in col or 'matmorb' in col]

    print(f"\nHealth outcome columns: {health_cols}")

    health_counts = {}
    for col in health_cols:
        health_counts[col] = article_df[col].value_counts().get(1, 0)

    print("Health outcome counts:")
    for col, count in sorted(health_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {col}: {count}")

    # Explore healthcare access and treatment columns
    access_cols = ['access_to_care', 'treatment_received']

    print("\nHealthcare access and treatment columns:")
    for col in access_cols:
        print(f"  {col}: {article_df[col].value_counts().get(1, 0)} studies")

    # Explore model data
    print("\nModel data:")
    print(f"  Number of unique effect size measures: {model_df['measure'].nunique()}")
    print(f"  Most common effect size measures: {model_df['measure'].value_counts().head(5).to_dict()}")

    # Check effect size columns
    effect_cols = ['point', 'lower', 'upper']
    print("\nEffect size columns - missing values:")
    for col in effect_cols:
        missing = model_df[col].isnull().sum()
        print(f"  {col}: {missing} missing values ({missing/len(model_df)*100:.2f}%)")

    # Check negative values in confidence intervals (potential data coding issue)
    for col in ['lower', 'upper']:
        neg_values = (model_df[col] < 0).sum()
        print(f"  {col}: {neg_values} negative values ({neg_values/len(model_df)*100:.2f}%)")

    return None

def merge_datasets(article_df, model_df):
    """
    Merge article and model datasets on DOI
    """
    print("\n--- Merging Datasets ---")

    # Merge on DOI
    df = pd.merge(model_df, article_df, on='doi', how='left')
    print(f"Merged dataset shape: {df.shape}")

    # Check if merge was successful
    if len(df) == len(model_df):
        print("All model records successfully matched with article data")
    else:
        print(f"Warning: {len(model_df) - len(df)} model records couldn't be matched")

    return df

import pandas as pd
import re
from rapidfuzz import process


def handle_compound_race(label):
    """
    Standardizing race labels
    """
    print("\n--- Standardizing race labels ---")
    if pd.isna(label):
        return 'Unknown / Not Reported'

    label = label.lower().strip()
    label = re.sub(r'[;:,_]', '/', label)
    label = re.sub(r'\s*(and|or|/)\s*', '/', label)
    parts = [p.strip() for p in label.split('/') if p.strip()]

    matched = set()
    for part in parts:
        if part in race_standardization:
            matched.add(race_standardization[part])
        else:
            match, score, _ = process.extractOne(part, race_standardization.keys())
            if score >= 90:
                matched.add(race_standardization[match])

    if not matched:
        return 'Other / Multiracial'
    elif len(matched) == 1:
        return list(matched)[0]
    else:
        return 'Other / Multiracial'

def standardize_race_labels(series, use_fuzzy=True, fuzzy_threshold=90):
    def normalize(label):
        if pd.isna(label):
            return 'unknown'
        return label.strip().lower()

    def map_label(label):
        if label in race_standardization:
            return race_standardization[label]
        if use_fuzzy:
            match, score, _ = process.extractOne(label, race_standardization.keys())
            if score >= fuzzy_threshold:
                return race_standardization[match]
        return handle_compound_race(label)

    normalized_series = series.apply(normalize)
    standardized_series = normalized_series.apply(map_label)

    original_unmapped = series[~normalized_series.isin(race_standardization.keys())].unique()
    if len(original_unmapped) > 0:
        print("\n⚠️ Unmapped race labels for review:")
        for label in original_unmapped:
            print(f"  - {label}")

    return standardized_series


def analyze_missing_data(df):
    """
    Analyze missing data patterns in the dataset
    """
    print("\n--- Missing Data Analysis ---")

    # Calculate missing data percentages
    missing_data = df.isnull().mean().sort_values(ascending=False) * 100
    print("Columns with highest percentage of missing values:")
    print(missing_data[missing_data > 0].head(10))

    # Check if imputation is needed
    effect_cols = ['point', 'lower', 'upper']
    missing_pct = df[effect_cols].isnull().mean() * 100
    print(f"\nMissing data in effect size columns: {missing_pct.to_dict()}")

    if missing_pct.max() < 20:
        print("Missing data percentage is below 20%, imputation can be applied")
        return True
    else:
        print("High percentage of missing data, analysis will proceed with complete cases only")
        return False

def impute_missing_data(df, columns_to_impute):
    """
    Impute missing values using KNN imputation
    """
    print("\n--- Imputing Missing Data ---")

    # Create imputer
    imputer = KNNImputer(n_neighbors=5)

    # Get subset of data for imputation
    df_to_impute = df[columns_to_impute].copy()

    # Standardize features
    scaler = StandardScaler()
    df_to_impute_scaled = pd.DataFrame(
        scaler.fit_transform(df_to_impute),
        columns=df_to_impute.columns
    )

    # Perform imputation
    print(f"  Imputing {columns_to_impute} using KNN (k=5)")
    df_imputed_scaled = pd.DataFrame(
        imputer.fit_transform(df_to_impute_scaled),
        columns=df_to_impute.columns
    )

    # Reverse scaling
    df_imputed = pd.DataFrame(
        scaler.inverse_transform(df_imputed_scaled),
        columns=df_to_impute.columns
    )

    # Create output dataframe
    result_df = df.copy()
    result_df[columns_to_impute] = df_imputed

    # Report results
    for col in columns_to_impute:
        before = df[col].isnull().sum()
        after = result_df[col].isnull().sum()
        print(f"  {col}: {before} missing values before, {after} missing values after imputation")

    return result_df

## Race standardization mapping

In [7]:
# Lowercased standardized mapping
race_standardization = {
    'african american': 'Black or African American',
    'african american or black': 'Black or African American',
    'african american, non-hispanic': 'Black or African American',
    'black': 'Black or African American',
    'black or african american': 'Black or African American',
    'black, non-hispanic': 'Black or African American',
    'non african american': 'Black or African American',
    'non-african american': 'Black or African American',
    'non-hispanic black': 'Black or African American',

    'caucasian': 'White',
    'white': 'White',
    'non hispanic white': 'White',
    'non-hispanic white': 'White',
    'non-latina white': 'White',
    'white (non-hispanic)': 'White',
    'white non-hispanic': 'White',
    'white/caucasian': 'White',
    'whites': 'White',
    'u.s.-born, non-hispanic white;': 'White',
    'non-hispanic, white': 'White',
    'white donor / white recipient': 'White',
    'european american': 'White',
    'non hispanic whites': 'White',
    'white, non-hispanic': 'White',

    'hispanic': 'Hispanic or Latino',
    'latina': 'Hispanic or Latino',
    'latino': 'Hispanic or Latino',
    'mexican/american': 'Hispanic or Latino',

    'asian': 'Asian or Pacific Islander',
    'asian, non-hispanic': 'Asian or Pacific Islander',
    'asian american': 'Asian or Pacific Islander',
    'pacific islander': 'Asian or Pacific Islander',

    'american indian': 'American Indian or Alaska Native',
    'american indian/alaska native': 'American Indian or Alaska Native',
    'american indian/alaskan': 'American Indian or Alaska Native',
    'american indian/alaskan native': 'American Indian or Alaska Native',
    'native american': 'American Indian or Alaska Native',
    'non-hispanic american indian or alaska native': 'American Indian or Alaska Native',

    'minority': 'Other / Multiracial',
    'nonwhite': 'Other / Multiracial',
    'non-white': 'Other / Multiracial',
    'urm (black, hispanic, native american/alaskan, asian/pacific islander, and other)': 'Other / Multiracial',
    'other': 'Other / Multiracial',
    'multiracial': 'Other / Multiracial',

    'unknown': 'Unknown / Not Reported',
    'not reported': 'Unknown / Not Reported'
}

## Main block to run preprocessing pipeline

In [10]:
# Set file paths
article_path = 'article_dat.csv'
model_path = 'model_dat.csv'


article_df, model_df = load_data(article_path, model_path)
explore_data(article_df, model_df)
df = merge_datasets(article_df, model_df)
should_impute = analyze_missing_data(df)

    
if should_impute:
    effect_cols = ['point', 'lower', 'upper']
    df = impute_missing_data(df, effect_cols)



print("\n=== ANALYSIS COMPLETE ===")



Loading datasets...
Article dataset shape: (318, 65)
Model dataset shape: (6804, 16)

--- Data Exploration ---

Article data - column types:
float64    37
object     24
int64       4
Name: count, dtype: int64

Health outcome columns: ['health_outcome', 'cancer_ovarian', 'cancer_uterine', 'cancer_cervical', 'cancer_vulvar', 'endo', 'fibroids', 'fert', 'matmorbmort']
Health outcome counts:
  health_outcome: 239
  matmorbmort: 94
  cancer_uterine: 69
  cancer_ovarian: 50
  fert: 31
  cancer_cervical: 28
  cancer_vulvar: 10
  fibroids: 8
  endo: 2

Healthcare access and treatment columns:
  access_to_care: 119 studies
  treatment_received: 184 studies

Model data:
  Number of unique effect size measures: 83
  Most common effect size measures: {'OR': 1975, 'Percent': 1370, 'RR': 1200, 'HR': 679, 'Incidence': 225}

Effect size columns - missing values:
  point: 36 missing values (0.53%)
  lower: 235 missing values (3.45%)
  upper: 237 missing values (3.48%)
  lower: 2081 negative values (30.

# Question 1: How are race and ethnicity categorized in medical research?

In [13]:
import plotly.express as px
import dash
from dash import dcc, html, Input, Output, ctx
from wordcloud import WordCloud
import numpy as np
import matplotlib.pyplot as plt
import base64
from io import BytesIO
import plotly.graph_objects as go

In [46]:
race_cols = [f"race{i}" for i in range(1, 9)]
ss_cols = [f"race{i}_ss" for i in range(1, 9)]
Q1_df=article_df.copy()
Q1_df.loc[:, ss_cols] = Q1_df[ss_cols].replace(-99, np.nan)

In [48]:
# Select only the desired columns
columns_to_keep = [
    "pmid", "doi", "jabbrv", "journal", "year", "month", "day", 
    "title", "abstract", "keywords", "study_aim", "race1", "race1_ss"
]

Q1_df = Q1_df[columns_to_keep]

In [50]:
Q1_df['race_group'] = standardize_race_labels(Q1_df['race1'])


--- Standardizing race labels ---

⚠️ Unmapped race labels for review:
  - White - not Hispanic or Latino
  - European Americans
  - Black/African American
  - Caucasians
  - Non-Hispanic Asian
  - White (or Caucasian)
  - Unavailable
  - Hispanic, Latina
  - White-non-Hispanic


In [52]:
# Aggregate article counts per race_group and year
race_group_counts = Q1_df.groupby(["year", "race_group"]).size().reset_index(name="article_count")

min_year, max_year = int(df["year"].min()), int(df["year"].max())
# Get a fixed list of all race groups
all_race_groups = sorted(Q1_df["race_group"].unique())

# Initialize Dash app
app = dash.Dash(__name__)


app.layout = html.Div([
    dcc.Graph(id="race-group-line-chart", style={"width": "100%", "margin": "0 auto", "display": "block"}),
    
    html.Div([
        dcc.Slider(
            id="year-slider",
            min=min_year,
            max=max_year,
            value=max_year,  # Default to most recent year
            marks={year: str(year) for year in range(min_year, max_year + 1)},
            step=1
        ),
        html.Button("Reset", id="reset-button", n_clicks=0),
    ], id="slider-btn-div", style={"width": "50%", "margin": "0 auto", "textAlign": "center"}),
    
    html.Div([
        dcc.Graph(id="race-group-bar-chart", style={"width": "50%", "display": "inline-block"}),
        html.Img(id="word-cloud", style={"width": "50%", "display": "inline-block"}),
    ], style={"display": "flex", "justify-content": "center"}),
    html.Div()
])

@app.callback(
    [Output("race-group-bar-chart", "figure"),
     Output("race-group-line-chart", "figure"),
     Output("year-slider", "value")],  # Allow reset button to reset slider value
    [Input("year-slider", "value"),
     Input("reset-button", "n_clicks")]
)
def update_charts(selected_year, reset_clicks):
    triggered_id = ctx.triggered_id if ctx.triggered_id else "year-slider"
    
    if triggered_id == "reset-button" or not selected_year:
        filtered_df = race_group_counts.groupby("race_group")["article_count"].sum().reset_index()
        bar_title = "Number of Articles by Race Group (All Years)"
        bar_chart = px.bar(filtered_df, x="race_group", y="article_count", title=bar_title)
        selected_year = max_year  # Reset slider
    else:
        filtered_df = race_group_counts[race_group_counts["year"] == selected_year]
        full_data = pd.DataFrame({"race_group": all_race_groups})
        filtered_df = full_data.merge(filtered_df, on="race_group", how="left").fillna(0)
        filtered_df["article_count"] = filtered_df["article_count"].astype(int)
        bar_title = f"Number of Articles by Race Group ({selected_year})"
        bar_chart = px.bar(filtered_df, x="race_group", y="article_count", title=bar_title)
    
    line_chart = px.line(race_group_counts, x="year", y="article_count", color="race_group", title="Articles Over Time by Race Group")
    line_chart.update_layout(legend=dict(font=dict(size=10)))  # Reduce legend font size
    
    return bar_chart, line_chart, selected_year

@app.callback(
    Output("word-cloud", "src"),
    Input("race-group-bar-chart", "clickData")
)
def update_word_cloud(click_data):
    if click_data is None:
        return None
    
    selected_race_group = click_data["points"][0]["x"]
    filtered_titles = Q1_df[Q1_df["race_group"] == selected_race_group]["title"]
    text = " ".join(filtered_titles)
    
    wordcloud = WordCloud(width=400, height=300, background_color='white').generate(text)
    img = BytesIO()
    wordcloud.to_image().save(img, format="PNG")
    encoded_img = base64.b64encode(img.getvalue()).decode()
    
    return f"data:image/png;base64,{encoded_img}"


if __name__ == "__main__":
    app.run(port=8021,debug=True)

# Question 2: What health outcomes have been studied, and what disparities have been identified?

In [24]:
# Broad categories and associated keywords to classify outcomes
category_keywords = {
    'Maternal': ['maternal', 'cesarean', 'pregnancy-related mortality', 'hysterectomy', 'uterine', 'pregnancy', 'transfusion', 'ICU', 'perineal', 'postpartum'],
    'Neonatal': ['neonatal', 'birthweight', 'NICU', 'apgar', 'infant', 'small for gestational age', 'preterm'],
    'Fertility / ART': ['fertility', 'ivf', 'in vitro', 'oocyte', 'assisted reproductive', 'cryopreservation', 'gonadotropin', 'cycle', 'embryo'],
    'Cancer': ['cancer', 'chemotherapy', 'oncology', 'tumor', 'nccn', 'radiotherapy', 'survival'],
    'Mental Health': ['depression', 'stress', 'mental health'],
    'COVID-19': ['covid', 'sars-cov-2', 'coronavirus'],
    'Access & Experience': ['insurance', 'access', 'care', 'pain', 'disparity', 'recommendation letters'],
    'Other': []  # default category for anything not matching above
}

# Function to categorize outcomes
def categorize_outcome(outcome):
    outcome_lower = str(outcome).lower()
    for category, keywords in category_keywords.items():
        if any(re.search(rf"\b{kw}\b", outcome_lower) for kw in keywords):
            return category
    return 'Other'

# Combine and deduplicate all outcomes
unique_outcomes = pd.Series(df['outcome'].dropna().tolist() + df['health_outcome'].dropna().tolist()).dropna()
df['health_category'] = unique_outcomes.apply(categorize_outcome)

In [26]:
clean_df = df.copy()
# Create flags for health outcome-related columns
print("  Creating health outcome indicator variables")

# 1. Main indicators
for col in ['health_outcome', 'access_to_care', 'treatment_received']:
    if col in clean_df.columns:
        clean_df[f'has_{col}'] = clean_df[col].apply(lambda x: 1 if x == 1 else 0)

# 2. Race/ethnicity indicators based on 'compare' column
print("  Creating race/ethnicity comparison indicators")

clean_df['is_black_comparison'] = clean_df['compare'].str.contains(
    'black|Black|African', case=False, na=False).astype(int)

clean_df['is_hispanic_comparison'] = clean_df['compare'].str.contains(
    'hispanic|latina|latino', case=False, na=False).astype(int)

clean_df['is_asian_comparison'] = clean_df['compare'].str.contains(
    'asian|pacific', case=False, na=False).astype(int)

clean_df['is_indigenous_comparison'] = clean_df['compare'].str.contains(
    'native|indian|indigenous|american indian|alaska', case=False, na=False).astype(int)

# 3. Log-transform effect sizes for analysis
clean_df['log_point'] = np.log(clean_df['point'].replace(0, np.nan))

  Creating health outcome indicator variables
  Creating race/ethnicity comparison indicators


C:\Users\sheepie\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log



In [28]:
"""
Create datasets for health outcome analysis
"""
print("\n--- Creating Analysis Datasets ---")

# Create health outcome dataset
health_outcome_df = clean_df[clean_df['health_outcome'] == 1].copy()
print(f"Health outcome dataset: {health_outcome_df.shape[0]} records")

# Create dataset for access to care analysis
access_df = clean_df[clean_df['access_to_care'] == 1].copy()
print(f"Access to care dataset: {access_df.shape[0]} records")

# Create dataset for treatment received analysis
treatment_df = clean_df[clean_df['treatment_received'] == 1].copy()
print(f"Treatment received dataset: {treatment_df.shape[0]} records")

# Create dataset for both access and treatment
access_treatment_df = clean_df[(clean_df['access_to_care'] == 1) &
                             (clean_df['treatment_received'] == 1)].copy()
print(f"Access and treatment dataset: {access_treatment_df.shape[0]} records")




--- Creating Analysis Datasets ---
Health outcome dataset: 5212 records
Access to care dataset: 3022 records
Treatment received dataset: 3438 records
Access and treatment dataset: 1894 records


## 2.1. Descriptive Analysis: What health outcomes have been studied?

In [42]:
from dash import Dash, html, dcc, Input, Output

df['race1_clean'] = standardize_race_labels(df['race1'])
app = Dash(__name__)

# --- Preprocess data for dropdown and plots ---
all_races = ['All'] + sorted(df['race1_clean'].dropna().unique().tolist())

category_counts_all = df['health_category'].value_counts().reset_index()
category_counts_all.columns = ['health_category', 'count']

# --- Layout ---
app.layout = html.Div([
    html.H2("Health Outcome Categories by Race"),

    html.Div([
        html.Label("Select Racial Group:"),
        dcc.Dropdown(
            id='race-selector',
            options=[{'label': race, 'value': race} for race in all_races],
            value='All',
            clearable=False
        )
    ], style={'width': '30%', 'marginBottom': '20px'}),

    dcc.Graph(id='category-bubble-chart'),

    html.H4("Top 10 Health Outcomes in Selected Category"),
    dcc.Graph(id='top-outcomes-bar'),

    html.H4("Heatmap of Health Category Prevalence by Race"),
    dcc.Graph(id='heatmap')
])

# --- Callbacks ---
@app.callback(
    Output('category-bubble-chart', 'figure'),
    Input('race-selector', 'value')
)
def update_bubble_chart(selected_race):
    if selected_race == 'All':
        data = df
    else:
        data = df[df['race1_clean'] == selected_race]

    category_counts = data[data['health_category'] != 'Other']['health_category'].value_counts().reset_index()
    category_counts.columns = ['health_category', 'count']

    fig = px.scatter(
        category_counts,
        x='health_category', y=[1]*len(category_counts),
        size='count', color='count', text='health_category',
        size_max=100, height=400, color_continuous_scale='Viridis_r'
    )
    fig.update_traces(textposition='middle center')
    fig.update_layout(
        showlegend=False,
        xaxis_title='', yaxis_title='',
        yaxis=dict(showticklabels=False),
        title="Prevalence of Health Outcome Categories"
    )
    return fig

@app.callback(
    Output('top-outcomes-bar', 'figure'),
    Input('race-selector', 'value')
)
def update_outcomes_bar(selected_race):
    if selected_race == 'All':
        data = df
    else:
        data = df[df['race1_clean'] == selected_race]

    outcome_counts = data['outcome'].value_counts().head(10).reset_index()
    outcome_counts.columns = ['outcome', 'count']
    outcome_counts = outcome_counts.sort_values('count', ascending=True)

    fig = px.bar(
        outcome_counts,
        x='count', y='outcome', orientation='h',
        color='count', color_continuous_scale='Viridis_r'
    )
    fig.update_layout(title="Top 10 Health Outcomes", yaxis_title="Outcome", xaxis_title="Count")
    return fig



@app.callback(
    Output('heatmap', 'figure'),
    Input('race-selector', 'value')
)
def update_heatmap(selected_race):
    data = df[df['health_category'] != 'Other']

    if selected_race != 'All':
        data = data[data['race1_clean'] == selected_race]

    heatmap_data = data.groupby(['race1_clean', 'health_category']).size().reset_index(name='count')
    pivot = heatmap_data.pivot(index='race1_clean', columns='health_category', values='count').fillna(0)

    fig = px.imshow(
        pivot,
        text_auto=True,
        labels=dict(x="Health Category", y="Race", color="Count"),
        title="Heatmap of Health Category Prevalence by Race",
        aspect="auto",
        height=500
    )
    return fig

# --- Run App ---
if __name__ == '__main__':
    app.run(port=8022, debug=True)


## 2.2 Compare effect sizes and visualize disparities between different race groups

In [34]:

# Initialize Dash app
app = dash.Dash(__name__)

# Define effect size measures
common_measures = ['OR', 'RR', 'HR']
effect_df = health_outcome_df[health_outcome_df['measure'].isin(common_measures)].copy()

# Log-transform effect sizes
effect_df['log_point'] = np.log(effect_df['point'].replace(0, np.nan))

# Define demographic groups
demo_groups = {
    'is_black_comparison': 'Black/African American',
    'is_hispanic_comparison': 'Hispanic/Latino',
    'is_asian_comparison': 'Asian/Pacific Islander',
    'is_indigenous_comparison': 'Indigenous/Native'
}

# Dropdown options
race_options = [{"label": name, "value": key} for key, name in demo_groups.items()]

# Identify top outcomes
top_outcomes = effect_df['outcome'].value_counts().head(5).index.tolist()

# App layout with vertical stacking
app.layout = html.Div([
    html.H2("Effect Sizes by Demographic Group", style={"textAlign": "center"}),

    dcc.Dropdown(
        id="group-selector",
        options=race_options,
        value="is_black_comparison",
        style={"width": "50%", "margin": "0 auto 30px auto"}
    ),

    html.Div([
        html.H4("Violin Plot: Log Effect Size Distribution", style={"textAlign": "center"}),
        html.Img(id="violin-plot", style={"width": "80%", "margin": "auto", "display": "block", "marginBottom": "50px"}),
    ]),

    html.Div([
        html.H4("Forest Plot: Top Health Outcomes", style={"textAlign": "center"}),
        html.Img(id="forest-plot", style={"width": "80%", "margin": "auto", "display": "block"})
    ])
])

# Callback to update both plots
@app.callback(
    [Output("violin-plot", "src"),
     Output("forest-plot", "src")],
    Input("group-selector", "value")
)
def update_plots(group_key):
    group_name = demo_groups[group_key]
    group_data = effect_df[effect_df[group_key] == 1].copy()

    #### --- VIOLIN PLOT --- ####
    violin_src = None
    if not group_data.empty:
        plt.figure(figsize=(10, 8))
        ax = sns.violinplot(x='measure', y='log_point', data=group_data)
        ax.axhline(y=0, color='r', linestyle='--')
        ax.set_title(f'{group_name}')
        ax.set_ylabel('Log Effect Size')
        ax.set_xlabel('Effect Measure')
        for i, measure in enumerate(group_data['measure'].unique()):
            count = len(group_data[group_data['measure'] == measure])
            ax.text(i, ax.get_ylim()[1]*0.9, f'n={count}', ha='center')
        buf_v = BytesIO()
        plt.tight_layout()
        plt.savefig(buf_v, format="png")
        plt.close()
        violin_src = f"data:image/png;base64,{base64.b64encode(buf_v.getvalue()).decode()}"

    #### --- FOREST PLOT --- ####
    forest_src = None
    forest_data = group_data[
        group_data['outcome'].isin(top_outcomes) &
        (~group_data['point'].isnull()) &
        (~group_data['lower'].isnull()) &
        (~group_data['upper'].isnull())
    ]

    forest_stats = []
    for outcome in top_outcomes:
        for measure in ['OR', 'RR']:
            subset = forest_data[
                (forest_data['outcome'] == outcome) &
                (forest_data['measure'] == measure)
            ]
            if len(subset) >= 3:
                forest_stats.append({
                    "Outcome": outcome,
                    "Measure": measure,
                    "Count": len(subset),
                    "Point": subset["point"].median(),
                    "Lower": subset["lower"].median(),
                    "Upper": subset["upper"].median()
                })

    if forest_stats:
        forest_df = pd.DataFrame(forest_stats).sort_values(['Outcome', 'Measure', 'Point'])
        plt.figure(figsize=(11, len(forest_df) + 3))
        y_pos = np.arange(len(forest_df))
        plt.errorbar(
            x=forest_df['Point'],
            y=y_pos,
            xerr=[forest_df['Point'] - forest_df['Lower'], forest_df['Upper'] - forest_df['Point']],
            fmt='o', capsize=5
        )
        plt.yticks(y_pos, forest_df['Outcome'] + ' (' + forest_df['Measure'] + ', n=' + forest_df['Count'].astype(str) + ')')
        plt.axvline(x=1, color='r', linestyle='--')
        plt.xscale('log')
        plt.xlabel('Effect Size (OR/RR)')
        plt.title(f'{group_name}')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        buf_f = BytesIO()
        plt.savefig(buf_f, format="png")
        plt.close()
        forest_src = f"data:image/png;base64,{base64.b64encode(buf_f.getvalue()).decode()}"

    return violin_src, forest_src

if __name__ == "__main__":
    app.run(debug=True, port=8023)

C:\Users\sheepie\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning:

invalid value encountered in log

